# Pyteal Working Examples

Pyteal is a python library for Teal to help algorand developers to get easy and smooth developing experiences. It can work on its own or together with Python SDK.

### A simple pyteal program

In [8]:
from pyteal import *

OPTIONS = CompileOptions(version=2)

""" fee is less than 300
"""
cond = Txn.fee() < Int(300)
print(cond.__teal__(OPTIONS))

(TealSimpleBlock([TealOp((Txn Fee), txn, 'Fee')], next=TealSimpleBlock([TealOp((Int: 300), int, 300)], next=TealSimpleBlock([TealOp((< (Txn Fee) (Int: 300)), <)], next=None))), TealSimpleBlock([TealOp((< (Txn Fee) (Int: 300)), <)], next=None))


### Oops, type error

In [5]:
from pyteal import *
""" type error
"""
cond = Txn.fee() < Txn.receiver()

TealTypeError: TealType.bytes while expected TealType.uint64 

### Hased Time Locked Contract

In [11]:
from pyteal import *

""" Hashed Time Locked Contract
"""
alice = Addr("6ZHGHH5Z5CTPCF5WCESXMGRSVK7QJETR63M3NY5FJCUYDHO57VTCMJOBGY")
bob = Addr("7Z5PWO2C6LFNQFGHWKSK5H47IQP5OJW2M3HA2QPXTY3WTNP5NU2MHBW27M")
secret = Bytes("base32", "2323232323232323")

fee_cond = Txn.fee() < Int(300)
type_cond = Txn.type_enum() == Int(1)
recv_cond = And(Txn.close_remainder_to() == Global.zero_address(),
                Txn.receiver() == alice,
                Sha256(Arg(0)) == secret)
esc_cond = And(Txn.close_remainder_to()  == Global.zero_address(),
               Txn.receiver() == bob,
               Txn.first_valid() > Int(3000))

atomic_swap = And(fee_cond,
                  type_cond,
                  recv_cond.Or(esc_cond))   

print(atomic_swap.__teal__(OPTIONS))

(TealSimpleBlock([TealOp((Txn Fee), txn, 'Fee')], next=TealSimpleBlock([TealOp((Int: 300), int, 300)], next=TealSimpleBlock([TealOp((< (Txn Fee) (Int: 300)), <)], next=TealSimpleBlock([TealOp((Txn TypeEnum), txn, 'TypeEnum')], next=TealSimpleBlock([TealOp((Int: 1), int, 1)], next=TealSimpleBlock([TealOp((== (Txn TypeEnum) (Int: 1)), ==)], next=TealSimpleBlock([TealOp((&& (< (Txn Fee) (Int: 300)) (== (Txn TypeEnum) (Int: 1)) (|| (&& (== (Txn CloseRemainderTo) (Global ZeroAddress)) (== (Txn Receiver) (address: 6ZHGHH5Z5CTPCF5WCESXMGRSVK7QJETR63M3NY5FJCUYDHO57VTCMJOBGY)) (== (sha256 (arg 0)) (base32 bytes: 2323232323232323))) (&& (== (Txn CloseRemainderTo) (Global ZeroAddress)) (== (Txn Receiver) (address: 7Z5PWO2C6LFNQFGHWKSK5H47IQP5OJW2M3HA2QPXTY3WTNP5NU2MHBW27M)) (> (Txn FirstValid) (Int: 3000))))), &&)], next=TealSimpleBlock([TealOp((Txn CloseRemainderTo), txn, 'CloseRemainderTo')], next=TealSimpleBlock([TealOp((Global ZeroAddress), global, 'ZeroAddress')], next=TealSimpleBlock([TealO